In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

_In this training the following configurations are used:_
```transformers = [transform_year, transform_power, transform_capacity, transform_price]
fillers = [fill_engine, fill_insurance, fill_damage, fill_type, fill_model, fill_gearbox, fill_fuel]
merger = True```

_Columns:_

```['engine_capacity', 'type', 'registration_year', 'gearbox', 'power',
       'model', 'mileage', 'fuel', 'brand', 'damage', 'zipcode',
     'insurance_price', 'city', 'latitude', 'longitude']```
     
_Algorithm:_ **LightGBM**

_First result best params:_

lgbm-v1
```{
    'objective': 'regression_l2', 
    'num_leaves': 98, 
    'num_iterations': 1200, 
    'min_data_in_leaf': 36, 
    'log_target': True, # we log target twice 0_0
    'learning_rate': 0.018000000000000002, 
    'feature_fraction': 0.85, 
    'bagging_freq': 10, 
    'bagging_fraction': 0.75
}```

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  prepare/  review/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test_no_target.csv  train.csv  zipcodes.csv


In [5]:
%ls data/processed

filled_transformed_train.csv         transformed_train.csv
merged_filled_transformed_train.csv


In [6]:
data_path = 'data/processed/'
data_file = 'merged_filled_transformed_train.csv'

In [7]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [8]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price,city,latitude,longitude
0,2.000000,bus,2006,auto,140.00000,c4,150000,gasoline,citroen,0.0,49191,5.940171,8.358666,Belm,52.30476,8.12846
1,1.780292,limousine,2016,manual,121.10506,vito,150000,gasoline,mercedes_benz,0.0,45896,5.438079,7.806696,Gelsenkirchen,51.51750,7.08575
2,2.200000,limousine,2010,manual,175.00000,mondeo,125000,diesel,ford,0.0,59229,6.835185,9.247058,"Ahlen, Westfalen",51.75972,7.89694
3,2.768020,limousine,2000,auto,265.00000,andere,150000,gasoline,ford,0.0,39365,6.522093,8.867568,"Marienborn, Börde",52.19628,11.10652
4,1.697200,convertible,2003,manual,109.00000,2_reihe,150000,gasoline,peugeot,0.0,55271,5.438079,7.768533,Stadecken-Elsheim,49.91220,8.12528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1.400000,limousine,2006,manual,75.00000,golf,90000,gasoline,volkswagen,0.0,35745,6.214608,8.452335,"Herborn, Hessen",50.68330,8.31667
49996,1.300000,small car,2004,manual,60.00000,fiesta,150000,gasoline,ford,0.0,60386,5.438079,6.761573,Frankfurt am Main,50.11670,8.68333
49997,1.978633,limousine,1996,manual,150.00000,5er,150000,gasoline,bmw,0.0,28309,4.867534,7.729735,Bremen,53.07516,8.80777
49998,1.786435,limousine,2007,manual,122.00000,1er,100000,diesel,bmw,0.0,83623,6.214608,9.005037,Dietramszell,47.85000,11.60000


## Convert to category

In [9]:
cdata = data
for c in cdata.select_dtypes(include='object'):
    cdata[c] = cdata[c].astype('category')

In [10]:
cdata

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price,city,latitude,longitude
0,2.000000,bus,2006,auto,140.00000,c4,150000,gasoline,citroen,0.0,49191,5.940171,8.358666,Belm,52.30476,8.12846
1,1.780292,limousine,2016,manual,121.10506,vito,150000,gasoline,mercedes_benz,0.0,45896,5.438079,7.806696,Gelsenkirchen,51.51750,7.08575
2,2.200000,limousine,2010,manual,175.00000,mondeo,125000,diesel,ford,0.0,59229,6.835185,9.247058,"Ahlen, Westfalen",51.75972,7.89694
3,2.768020,limousine,2000,auto,265.00000,andere,150000,gasoline,ford,0.0,39365,6.522093,8.867568,"Marienborn, Börde",52.19628,11.10652
4,1.697200,convertible,2003,manual,109.00000,2_reihe,150000,gasoline,peugeot,0.0,55271,5.438079,7.768533,Stadecken-Elsheim,49.91220,8.12528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1.400000,limousine,2006,manual,75.00000,golf,90000,gasoline,volkswagen,0.0,35745,6.214608,8.452335,"Herborn, Hessen",50.68330,8.31667
49996,1.300000,small car,2004,manual,60.00000,fiesta,150000,gasoline,ford,0.0,60386,5.438079,6.761573,Frankfurt am Main,50.11670,8.68333
49997,1.978633,limousine,1996,manual,150.00000,5er,150000,gasoline,bmw,0.0,28309,4.867534,7.729735,Bremen,53.07516,8.80777
49998,1.786435,limousine,2007,manual,122.00000,1er,100000,diesel,bmw,0.0,83623,6.214608,9.005037,Dietramszell,47.85000,11.60000


In [11]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   engine_capacity    50000 non-null  float64 
 1   type               50000 non-null  category
 2   registration_year  50000 non-null  int64   
 3   gearbox            50000 non-null  category
 4   power              50000 non-null  float64 
 5   model              50000 non-null  category
 6   mileage            50000 non-null  int64   
 7   fuel               50000 non-null  category
 8   brand              50000 non-null  category
 9   damage             50000 non-null  float64 
 10  zipcode            50000 non-null  int64   
 11  insurance_price    50000 non-null  float64 
 12  price              50000 non-null  float64 
 13  city               49946 non-null  category
 14  latitude           49946 non-null  float64 
 15  longitude          49946 non-null  float64 
dtypes: c

# Raw score

In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer

In [13]:
X, y = cdata.drop(['price'], axis=1), cdata['price']

In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
mape_scorer = make_scorer(mean_absolute_percentage_error , greater_is_better = False)

In [16]:
def hyperopt_lightgbm_basic(X, y, n_iter=50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=314)
    space = {
        'learning_rate': np.linspace(0.001, 0.05, num=50),
        'num_leaves': range(60, 100, 1),
        'num_iterations': range(700, 1500, 100),
        'min_data_in_leaf': range(10, 128, 1),
        'feature_fraction': np.linspace(0.7, 1.0, num=3),
        'bagging_fraction': np.linspace(0.5, 1.0, num=5),
        'bagging_freq': range(4, 20, 1),
        'log_target': [False, True],
        'objective': ['regression_l1', 'regression_l2', 'mape']
    }
    reg = lgb.LGBMRegressor(metric='mape', n_estimators=500)
    gs = RandomizedSearchCV(
        estimator=reg, 
        param_distributions=space, 
        n_iter=n_iter,
        scoring=mape_scorer,
        cv=5,
        refit=True,
        random_state=42,
        verbose=True)
    fit_params={"early_stopping_rounds":30, 
            "eval_set" : [(X_test, y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'categorical_feature': 'auto'}
    gs.fit(X_train, y_train, **fit_params)
    print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))
    return gs

In [17]:
best = {
    'objective': 'regression_l2', 
    'num_leaves': 98, 
    'num_iterations': 1200, 
    'min_data_in_leaf': 36, 
    'log_target': True, 
    'learning_rate': 0.018000000000000002, 
    'feature_fraction': 0.85, 
    'bagging_freq': 10, 
    'bagging_fraction': 0.75
}

In [18]:
if best is None:
    best = hyperopt_lightgbm_basic(X, y).best_params_

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=314)
reg = lgb.LGBMRegressor(metric='mape', n_estimators=500)
reg.fit(X_train, y_train)

LGBMRegressor(metric='mape', n_estimators=500)

In [20]:
reg.score(X_test, y_test)

0.9077010667200845

In [21]:
y_pred = reg.predict(X_test)

In [22]:
mean_absolute_percentage_error(y_test, y_pred)

3.0191026426494414

# Save model

In [34]:
import joblib
from datetime import datetime

In [44]:
model_path = 'models/'
model_file = f'lgbm-v1-{datetime.now()}.model'.replace(' ', '_')

In [45]:
joblib.dump(reg, model_path + model_file)

['models/lgbm-v1-2020-09-06_12:15:11.431861.model']